# Symmetric Eigenvalue Decomposition - Lanczos Method
---

If the matrix $A$ is large and sparse and/or if only some
eigenvalues and their eigenvectors are desired, iterative methods are
the methods of choice. For example, the power method can be useful to compute the
eigenvalue with the largest modulus. The basic
operation in the power method is matrix-vector multiplication, and this can be
performed very fast if $A$ is sparse. Moreover, $A$ need not be stored in the
computer --- the input for the algorithm can be just a function which,
given some vector $x$, returns the product $Ax$.

An _improved_ version of the power method, which efficiently computes
some eigenvalues (either largest in modulus or near some target value $\mu$)
and the corresponding eigenvectors, is the Lanczos method.

For more details, see 
[I. Slapničar, Symmetric Matrix Eigenvalue Techniques][Hog14]
and the references therein.

[Hog14]: #1 "L. Hogben, ed., 'Handbook of Linear Algebra', pp. 55.1-55.25, CRC Press, Boca Raton, 2014."


## Prerequisites

The reader should be familiar with concepts of eigenvalues and eigenvectors, related perturbation theory, and algorithms. 

 
## Competences 

The reader should be able to recognise matrices which warrant use uf Lanczos method, to apply 
the method and to assess the accuracy of the solution.

## Lanczos method

$A$ is a real symmetric matrix of order $n$.

### Definitions

Given a nonzero vector $x$ and an index $k<n$, the __Krylov matrix__ is defined as
$K_k=\begin{bmatrix} x & Ax & A^2 x &\cdots & A^{k-1}x \end{bmatrix}$.

__Krilov subspace__ is the subspace spanned by the columns of $K_k$.

### Facts

1. The Lanczos method is based on the following observation. If $K_k=XR$ is the
  $QR$ factorization of the matrix $K_k$, 
  then the $k\times k$ matrix $T=X^T A X$ is tridiagonal. The matrices $X$ and
  $T$ can be computed by using only matrix-vector products in $O(kn)$
  operations.

2. Let $T=Q\Lambda Q^T$ be the EVD of $T$. Then $\lambda_i$ approximate well some of the largest 
and smallest eigenvalues of $A$, and the columns of the matrix $U=XQ$ approximate the corresponding
  eigenvectors.

3. As $k$ increases, the largest (smallest) eigenvalues of the matrix
  $T_{1:k,1:k}$ converge towards some of the largest (smallest) eigenvalues of $A$ (due to
  the Cauchy interlace property). The algorithm can be redesigned to compute
  only largest or smallest eigenvalues. Also, by using shift and invert
  strategy, the method can be used to compute eigenvalues near some specified
  value. In order to obtain better approximations, $k$ should be greater than
  the number of required eigenvalues. On the other side, in order to obtain
  better accuracy and efficacy, $k$ should be as small as possible.

4. The last computed element, $\mu=T_{k+1,k}$, provides
  information about accuracy:
  \begin{align*}
  \|AU-U\Lambda\|_2&=\mu, \\
  \|AU_{:,i}-\lambda_i U_{:,i}\|_2&=\mu |Q_{ki}|, \quad  i=1,\ldots,k.
  \end{align*}
  Further, there are $k$
  eigenvalues $\tilde\lambda_1,\ldots,\tilde\lambda_k$ of $A$ such that
  $|\lambda_i-\tilde\lambda_i|\leq \mu$, and for the corresponding eigenvectors, we
  have $$\sin2\Theta(U_{:,i},\tilde U_{:,i}) \leq \frac{2\mu}{\min_{j\neq i} 
|\lambda_i-\tilde \lambda_j|}.$$ 

5. In practical implementations, $\mu$ is usually used to determine the index $k$. 

6. The Lanczos method has inherent
  numerical instability in the floating-point arithmetic: since the Krylov vectors are, in fact,
  generated by the power method, they converge towards an eigenvector of $A$. 
  Thus, as $k$ increases, the Krylov vectors become more and more parallel, and the recursion in the 
  function `myLanczos()` becomes numerically unstable and the computed columns of $X$
  cease to be sufficiently orthogonal. This affects both the convergence and
  the accuracy of the algorithm. For example, several eigenvalues of $T$ may
  converge towards a simple eigenvalue of $A$ (the, so
  called, _ghost eigenvalues_).

7. The loss of orthogonality is dealt with by using the __full
  reorthogonalization__ procedure: in each step, the new ${\bf z}$ is orthogonalized against all
previous
  columns of $X$, that is, in function `myLanczos()`, the formula 
  ```
  z=z-Tr.dv[i]*X[:,i]-Tr.ev[i-1]*X[:,i-1]
  ```
  is replaced 
  by
  ```
  z=z-sum(dot(z,Tr.dv[i])*X[:,i]-Tr.ev[i-1]*X[:,i-1]
  ```
  To obtain better orthogonality, the latter formula is usually executed twice. 
  The full reorthogonalization raises the operation count to $O(k^2n)$. 
8. The __selective reorthogonalization__ is the procedure in which the current $z$
  is orthogonalized against some selected columns of $X$, in order to
  attain sufficient numerical stability and not increase the operation count
  too much. The details are very subtle and can be found in the references.
  
9. The Lanczos method is usually used for sparse matrices. Sparse matrix $A$
  is stored in the sparse format in which only values and indices of nonzero elements
  are stored. The number of operations required to multiply some vector by $A$ is
  also proportional to the number of nonzero elements.
  
10. The function `eigs()` implements Lanczos method real for symmetric matrices and more general Arnoldi method 
for general matrices.

### Examples

In [1]:
function myLanczos{T}(A::Array{T}, x::Vector{T}, k::Int)
    n=size(A,1)
    X=Array(T,n,k)
    dv=Array(T,k)
    ev=Array(T,k-1)
    X[:,1]=x/norm(x)
    for i=1:k-1
        z=A*X[:,i]
        dv[i]=X[:,i]⋅z
        # Three-term recursion
        if i==1
            z=z-dv[i]*X[:,i]
        else
            # z=z-dv[i]*X[:,i]-ev[i-1]*X[:,i-1]
            # Full reorthogonalization - once or even twice
            z=z-sum([(z⋅X[:,j])*X[:,j] for j=1:i])
            # z=z-sum([(z⋅X[:,j])*X[:,j] for j=1:i])
        end
        μ=norm(z)
        if μ==0
            Tr=SymTridiagonal(dv[1:i-1],ev[1:i-2])
            return eigvals(Tr), X[:,1:i-1]*eigvecs(Tr), X[:,1:i-1], μ
        else
            ev[i]=μ
            X[:,i+1]=z/μ
        end
    end
    # Last step
    z=A*X[:,end]
    dv[end]=X[:,end]⋅z
    z=z-dv[end]*X[:,end]-ev[end]*X[:,end-1]
    μ=norm(z)
    Tr=SymTridiagonal(dv,ev)
    eigvals(Tr), X*eigvecs(Tr), X, μ
end

myLanczos (generic function with 1 method)

In [2]:
n=100
A=full(Symmetric(rand(n,n)))
# Or: A = rand(5,5) |> t -> t + t'
x=rand(n)
k=10

10

In [3]:
λ,U,X,μ=myLanczos(A,x,k)

([-5.37624,-4.42233,-3.26127,-1.77939,-0.00170013,1.84454,3.34942,4.33895,5.14009,49.6059],
[0.0198744 0.204573 … 0.122707 0.0985054; -0.183841 -0.0767038 … -0.0566093 0.0938514; … ; -0.0395262 -0.107676 … 0.10964 0.112808; -0.0601732 -0.0677798 … 0.124403 0.0957472],

[0.00688425 0.186215 … 0.114506 -0.169532; 0.102662 0.0173972 … -0.00452901 -0.11751; … ; 0.110905 0.0200454 … 0.00817994 0.0349199; 0.039973 0.112729 … 0.0240784 -0.132586],

2.538508177221226)

In [4]:
# Orthogonality
X'*X

10×10 Array{Float64,2}:
  1.0           2.93267e-17  -4.12699e-16  …   4.73165e-16   2.81661e-17
  2.93267e-17   1.0           3.51335e-16     -3.99485e-16  -1.58613e-16
 -4.12699e-16   3.51335e-16   1.0             -2.11874e-16   4.39235e-16
 -5.38537e-17   8.98859e-17  -5.22162e-16      6.16277e-16   2.32154e-16
  3.12164e-16  -3.23944e-16  -2.34218e-16     -4.27927e-16  -4.82806e-16
  1.08682e-16  -1.53891e-16   4.7568e-16   …  -5.71019e-16  -8.88398e-18
 -4.60994e-16   3.85788e-16   1.79579e-16     -1.26028e-16   4.31746e-16
 -7.12785e-17   1.35221e-16  -4.41669e-16      5.46529e-16  -1.00684e-18
  4.73165e-16  -3.99485e-16  -2.11874e-16      1.0          -4.46488e-16
  2.81661e-17  -1.58613e-16   4.39235e-16     -4.46488e-16   1.0        

In [5]:
X'*A*X

10×10 Array{Float64,2}:
 37.6399       21.2487       -6.67657e-15  …   7.54433e-15  -3.93664e-15
 21.2487       11.6029        3.66238          5.20938e-15   2.01572e-15
 -6.39994e-15   3.66238      -0.115481         3.92103e-16  -2.65196e-16
  6.36999e-16  -1.41117e-15   2.65477         -2.00156e-15  -4.09218e-16
  6.79562e-15   2.65726e-15   3.23498e-16      4.79757e-16   4.55429e-16
  1.47994e-15   1.92536e-15  -1.09739e-16  …  -1.66699e-15  -2.99295e-16
 -7.18449e-15  -3.20325e-15  -4.26454e-16      1.53875e-16   2.3496e-16 
  4.29371e-16  -1.93076e-15   3.49674e-16      2.59122       1.90665e-16
  9.22406e-15   4.14707e-15   5.49259e-17     -0.0277433     2.70551    
 -2.94606e-15   1.03643e-15   1.25005e-16      2.70551       0.532263   

In [6]:
# Residual
norm(A*U-U*diagm(λ)), μ

(2.538508177221227,2.538508177221226)

In [7]:
U'*A*U

10×10 Array{Float64,2}:
 -5.37624       6.46362e-16  -4.93325e-16  …   2.86873e-15   7.85996e-15
  7.66257e-16  -4.42233       3.93648e-16      5.66515e-15  -2.85846e-15
 -3.17507e-16   2.56534e-16  -3.26127          2.23747e-15  -1.28085e-14
 -3.41985e-15   1.03533e-15  -4.15588e-15      6.66143e-15  -3.18579e-15
 -1.59569e-16   1.82031e-15   4.28501e-15      2.87003e-15  -9.25253e-15
  2.37575e-15  -3.43336e-15   3.95453e-15  …  -3.13756e-15   1.62335e-14
 -3.1232e-15   -3.11538e-15   6.36454e-16      4.22239e-16  -7.29859e-15
  6.87733e-16  -1.76747e-15   1.00404e-15      1.23745e-14  -5.01984e-15
  3.256e-15     5.87686e-15   2.59287e-15      5.14009      -6.34375e-16
  8.52745e-15  -4.84355e-15  -1.29708e-14     -1.16837e-15  49.6059     

In [8]:
U'*U

10×10 Array{Float64,2}:
  1.0          -4.05137e-17   3.36542e-16  …  -1.24234e-16  -3.73413e-17
 -4.05137e-17   1.0           4.29276e-17     -1.41805e-17   1.35197e-16
  3.36542e-16   4.29276e-17   1.0              3.18237e-17   9.00492e-17
  1.99863e-16   2.24619e-16   4.83712e-16      2.81383e-16  -3.14696e-16
  2.61546e-16  -2.2164e-16    7.08503e-17      2.21723e-16   9.47957e-18
 -3.70841e-16   6.77529e-17  -2.55833e-16  …  -1.75056e-16   2.06278e-16
  9.71831e-17  -1.82876e-16  -5.31595e-16     -6.03296e-16   1.2687e-18 
 -2.27486e-16   1.48481e-16   2.99525e-16      1.33711e-15  -3.68457e-17
 -1.24234e-16  -1.41805e-17   3.18237e-17      1.0           4.47212e-17
 -3.73413e-17   1.35197e-16   9.00492e-17      4.47212e-17   1.0        

In [9]:
λeig,Ueig=eig(A);

In [10]:
?eigs

search: eigs eigvecs eigvals eigvals! leading_ones leading_zeros



```
eigs(A; nev=6, ncv=max(20,2*nev+1), which="LM", tol=0.0, maxiter=300, sigma=nothing, ritzvec=true, v0=zeros((0,))) -> (d,[v,],nconv,niter,nmult,resid)
```

Computes eigenvalues `d` of `A` using implicitly restarted Lanczos or Arnoldi iterations for real symmetric or general nonsymmetric matrices respectively.

The following keyword arguments are supported:

  * `nev`: Number of eigenvalues
  * `ncv`: Number of Krylov vectors used in the computation; should satisfy `nev+1 <= ncv <= n` for real symmetric problems and `nev+2 <= ncv <= n` for other problems, where `n` is the size of the input matrix `A`. The default is `ncv = max(20,2*nev+1)`. Note that these restrictions limit the input matrix `A` to be of dimension at least 2.
  * `which`: type of eigenvalues to compute. See the note below.

| `which` | type of eigenvalues                                                                                                       |
|:------- |:------------------------------------------------------------------------------------------------------------------------- |
| `:LM`   | eigenvalues of largest magnitude (default)                                                                                |
| `:SM`   | eigenvalues of smallest magnitude                                                                                         |
| `:LR`   | eigenvalues of largest real part                                                                                          |
| `:SR`   | eigenvalues of smallest real part                                                                                         |
| `:LI`   | eigenvalues of largest imaginary part (nonsymmetric or complex `A` only)                                                  |
| `:SI`   | eigenvalues of smallest imaginary part (nonsymmetric or complex `A` only)                                                 |
| `:BE`   | compute half of the eigenvalues from each end of the spectrum, biased in favor of the high end. (real symmetric `A` only) |

  * `tol`: parameter defining the relative tolerance for convergence of Ritz values (eigenvalue estimates).    A Ritz value $θ$ is considered converged when its associated residual    is less than or equal to the product of `tol` and $max(ɛ^{2/3}, |θ|)$,    where `ɛ = eps(real(eltype(A)))/2` is LAPACK's machine epsilon.    The residual associated with $θ$ and its corresponding Ritz vector $v$    is defined as the norm $||Av - vθ||$.    The specified value of `tol` should be positive; otherwise, it is ignored    and $ɛ$ is used instead.    Default: $ɛ$.
  * `maxiter`: Maximum number of iterations (default = 300)
  * `sigma`: Specifies the level shift used in inverse iteration. If `nothing` (default), defaults to ordinary (forward) iterations. Otherwise, find eigenvalues close to `sigma` using shift and invert iterations.
  * `ritzvec`: Returns the Ritz vectors `v` (eigenvectors) if `true`
  * `v0`: starting vector from which to start the iterations

`eigs` returns the `nev` requested eigenvalues in `d`, the corresponding Ritz vectors `v` (only if `ritzvec=true`), the number of converged eigenvalues `nconv`, the number of iterations `niter` and the number of matrix vector multiplications `nmult`, as well as the final residual vector `resid`.

!!! note
    The `sigma` and `which` keywords interact: the description of eigenvalues searched for by `which` do *not* necessarily refer to the eigenvalues of `A`, but rather the linear operator constructed by the specification of the iteration mode implied by `sigma`.

    | `sigma`         | iteration mode                   | `which` refers to eigenvalues of |
    |:--------------- |:-------------------------------- |:-------------------------------- |
    | `nothing`       | ordinary (forward)               | $A$                              |
    | real or complex | inverse with level shift `sigma` | $(A - \sigma I )^{-1}$           |


!!! note
    Although `tol` has a default value, the best choice depends strongly on the matrix `A`. We recommend that users _always_ specify a value for `tol` which suits their specific needs.

    For details of how the errors in the computed eigenvalues are estimated, see:

      * B. N. Parlett, "The Symmetric Eigenvalue Problem", SIAM: Philadelphia, 2/e (1998), Ch. 13.2, "Accessing Accuracy in Lanczos Problems", pp. 290-292 ff.
      * R. B. Lehoucq and D. C. Sorensen, "Deflation Techniques for an Implicitly Restarted Arnoldi Iteration", SIAM Journal on Matrix Analysis and Applications (1996), 17(4), 789–821.  doi:10.1137/S0895479895281484


```
eigs(A, B; nev=6, ncv=max(20,2*nev+1), which="LM", tol=0.0, maxiter=300, sigma=nothing, ritzvec=true, v0=zeros((0,))) -> (d,[v,],nconv,niter,nmult,resid)
```

Computes generalized eigenvalues `d` of `A` and `B` using implicitly restarted Lanczos or Arnoldi iterations for real symmetric or general nonsymmetric matrices respectively.

The following keyword arguments are supported:

  * `nev`: Number of eigenvalues
  * `ncv`: Number of Krylov vectors used in the computation; should satisfy `nev+1 <= ncv <= n` for real symmetric problems and `nev+2 <= ncv <= n` for other problems, where `n` is the size of the input matrices `A` and `B`. The default is `ncv = max(20,2*nev+1)`. Note that these restrictions limit the input matrix `A` to be of dimension at least 2.
  * `which`: type of eigenvalues to compute. See the note below.

| `which` | type of eigenvalues                                                                                                       |
|:------- |:------------------------------------------------------------------------------------------------------------------------- |
| `:LM`   | eigenvalues of largest magnitude (default)                                                                                |
| `:SM`   | eigenvalues of smallest magnitude                                                                                         |
| `:LR`   | eigenvalues of largest real part                                                                                          |
| `:SR`   | eigenvalues of smallest real part                                                                                         |
| `:LI`   | eigenvalues of largest imaginary part (nonsymmetric or complex `A` only)                                                  |
| `:SI`   | eigenvalues of smallest imaginary part (nonsymmetric or complex `A` only)                                                 |
| `:BE`   | compute half of the eigenvalues from each end of the spectrum, biased in favor of the high end. (real symmetric `A` only) |

  * `tol`: relative tolerance used in the convergence criterion for eigenvalues, similar to    `tol` in the [`eigs(A)`](:func:`eigs`) method for the ordinary eigenvalue    problem, but effectively for the eigenvalues of $B^{-1} A$ instead of $A$.    See the documentation for the ordinary eigenvalue problem in    [`eigs(A)`](:func:`eigs`) and the accompanying note about `tol`.
  * `maxiter`: Maximum number of iterations (default = 300)
  * `sigma`: Specifies the level shift used in inverse iteration. If `nothing` (default), defaults to ordinary (forward) iterations. Otherwise, find eigenvalues close to `sigma` using shift and invert iterations.
  * `ritzvec`: Returns the Ritz vectors `v` (eigenvectors) if `true`
  * `v0`: starting vector from which to start the iterations

`eigs` returns the `nev` requested eigenvalues in `d`, the corresponding Ritz vectors `v` (only if `ritzvec=true`), the number of converged eigenvalues `nconv`, the number of iterations `niter` and the number of matrix vector multiplications `nmult`, as well as the final residual vector `resid`.

**Example**

```julia
X = sprand(10, 5, 0.2)
eigs(X, nsv = 2, tol = 1e-3)
```

!!! note
    The `sigma` and `which` keywords interact: the description of eigenvalues searched for by `which` do *not* necessarily refer to the eigenvalue problem $Av = Bv\lambda$, but rather the linear operator constructed by the specification of the iteration mode implied by `sigma`.

    | `sigma`         | iteration mode                   | `which` refers to the problem  |
    |:--------------- |:-------------------------------- |:------------------------------ |
    | `nothing`       | ordinary (forward)               | $Av = Bv\lambda$               |
    | real or complex | inverse with level shift `sigma` | $(A - \sigma B )^{-1}B = v\nu$ |



In [11]:
λeigs,Ueigs=eigs(A; nev=k, which=:LM, ritzvec=true, v0=x)

([49.6059,-5.41433,-5.3566,5.2713,-5.14528,5.08176,-4.80509,4.8008,-4.59835,4.57263],
[-0.0985054 -0.00362906 … -0.0934272 -0.104632; -0.0938514 -0.10312 … 0.025156 0.0844137; … ; -0.112808 -0.107415 … -0.0237248 0.053201; -0.0957472 -0.0705846 … 0.113156 0.231745],

10,20,161,[-0.114912,0.315336,0.0408103,-0.251689,-0.0560213,0.416254,-0.272011,-0.268586,-0.155452,0.0517441  …  0.0207115,0.0746995,0.186193,-0.155403,-0.164821,-0.420773,0.494347,-0.0901784,0.161385,-0.0856163])

In [12]:
[sort(λ) sort(λeigs)]

10×2 Array{Float64,2}:
 -5.37624     -5.41433
 -4.42233     -5.3566 
 -3.26127     -5.14528
 -1.77939     -4.80509
 -0.00170013  -4.59835
  1.84454      4.57263
  3.34942      4.8008 
  4.33895      5.08176
  5.14009      5.2713 
 49.6059      49.6059 

In [13]:
sort(abs(λeig),rev=true)[1:k]

10-element Array{Float64,1}:
 49.6059 
  5.41433
  5.3566 
  5.2713 
  5.14528
  5.08176
  4.80509
  4.8008 
  4.59835
  4.57263

In [14]:
size(A)

(100,100)

We see that `eigs()` computes `k` eigenvalues with largest modulus. What eigenvalues did `myLanczos()` compute?

In [15]:
for i=1:k
    println(minabs(λeig-λ[i]))
end

0.019641737934624537
0.009737045688753554
0.01811269403537219
0.009630733461403196
0.03387739797066938
0.025994881973999684
0.04763117098326353
0.01709749286912121
0.058321334697038374
3.552713678800501e-14


Conslusion is that the simple implementation of Lanczos is not enough. However, it is fine, when all eigenvalues are computed:

In [16]:
λall,Uall,Xall,μall=myLanczos(A,x,100)

([-5.41433,-5.3566,-5.14528,-4.80509,-4.59835,-4.44685,-4.41259,-4.18043,-4.15725,-4.02166  …  4.1437,4.19962,4.32185,4.50682,4.54891,4.57263,4.8008,5.08176,5.2713,49.6059],
[-0.00362906 -0.0252182 … -0.00110532 0.0985054; -0.10312 0.133042 … 0.0154603 0.0938514; … ; -0.107415 -0.0520344 … 0.1306 0.112808; -0.0705846 0.0277077 … 0.0110164 0.0957472],

[0.00688425 0.186215 … 0.13216 -0.101628; 0.102662 0.0173972 … -0.0320799 0.027571; … ; 0.110905 0.0200454 … 0.146665 -0.0387435; 0.039973 0.112729 … 0.0124266 0.040965],

6.908007511927767e-14)

In [17]:
norm(A*Uall-Uall*diagm(λall)), norm(λeig-λall)

(1.5629825744393886e-13,4.3568735196250056e-14)

### Operator version

We can use Lanczos method with operator which, given vector `x`, returns the product `A*x`. We use the function `LinearMap()` from the package
[LinearMaps.jl](https://github.com/Jutho/LinearMaps.jl)

In [18]:
# Need Pkg.add("LinearMaps"); Pkg.checkout("LinearMaps")
using LinearMaps

In [19]:
methods(LinearMap)

# 2 methods for generic function "LinearMap":
LinearMap{T}(A::Union{AbstractArray{T,2},LinearMaps.AbstractLinearMap{T}}; kwargs...) at C:\Users\Ivan\.julia\v0.5\LinearMaps\src\LinearMaps.jl:63
LinearMap(args...; kwargs...) at C:\Users\Ivan\.julia\v0.5\LinearMaps\src\LinearMaps.jl:64

In [20]:
# Operator from the matrix
C=LinearMap(A)

LinearMaps.WrappedMap{Float64,Array{Float64,2}}([0.86254 0.914805 … 0.799886 0.791083; 0.914805 0.0657009 … 0.829267 0.00958788; … ; 0.799886 0.829267 … 0.677524 0.738646; 0.791083 0.00958788 … 0.738646 0.790206],true,true,true,false)

In [21]:
λC,UC=eigs(C; nev=k, which=:LM, ritzvec=true, v0=x)
λeigs-λC

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Here is an example of `LinearMap()` with the function. 

In [22]:
f(x)=A*x

f (generic function with 1 method)

In [23]:
D=LinearMap(f,n,issymmetric=true)

FunctionMap{Float64}(f, nothing, 100, 100; ismutating=false, issymmetric=true, ishermitian=true, isposdef=false)

In [24]:
λD,UD=eigs(D, nev=k, which=:LM, ritzvec=true, v0=x)
λeigs-λD

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

### Sparse matrices

In [25]:
?sprand

search: sprand sprandn sprandbool StepRange



```
sprand([rng],[type],m,[n],p::AbstractFloat,[rfn])
```

Create a random length `m` sparse vector or `m` by `n` sparse matrix, in which the probability of any element being nonzero is independently given by `p` (and hence the mean density of nonzeros is also exactly `p`). Nonzero values are sampled from the distribution specified by `rfn` and have the type `type`. The uniform distribution is used in case `rfn` is not specified. The optional `rng` argument specifies a random number generator, see [Random Numbers](:ref:`Random Numbers <random-numbers>`).


In [26]:
C=sprand(n,n,0.05) |> t -> t+t'

100×100 sparse matrix with 962 Float64 nonzero entries:
	[30 ,   1]  =  0.733823
	[36 ,   1]  =  0.963425
	[41 ,   1]  =  0.0496471
	[47 ,   1]  =  0.71481
	[49 ,   1]  =  0.723953
	[63 ,   1]  =  0.167829
	[68 ,   1]  =  0.312554
	[75 ,   1]  =  0.649626
	[81 ,   1]  =  0.81591
	[88 ,   1]  =  0.473922
	⋮
	[94 ,  99]  =  0.489814
	[5  , 100]  =  0.65318
	[12 , 100]  =  0.281851
	[14 , 100]  =  0.344055
	[23 , 100]  =  0.326371
	[25 , 100]  =  0.349243
	[46 , 100]  =  0.977554
	[55 , 100]  =  0.00743095
	[63 , 100]  =  0.027197
	[67 , 100]  =  0.542983
	[94 , 100]  =  0.106635

In [27]:
issymmetric(C)

true

In [28]:
eigs(C; nev=k, which=:LM, ritzvec=true, v0=x)

([6.06623,3.95313,-3.69512,3.67327,-3.41892,3.39165,-3.2382,3.20934,-3.17253,-3.07507],
[0.111254 0.0691826 … -0.122346 0.201909; 0.120358 0.147051 … -0.0454013 -0.0698497; … ; 0.149604 -0.0528931 … -0.121894 0.115316; 0.0468502 0.017566 … -0.0413385 -0.0544012],

10,13,116,[-0.261792,0.0205895,-0.223273,0.0137784,-0.129588,0.0910415,0.117277,-0.0780567,0.180835,-0.243295  …  -0.34771,-0.120799,-0.219801,-0.297889,0.0365402,0.10174,-0.250802,0.110493,0.0848316,0.0436877])